# Daily Energy

The most basic piece of information is the daily energy use from each microgrid.  The meter provides a kWh export field that we can sample on a daily basis to get this information.

I'm building on earlier work in December so that I can analyze multiple villages at once and make comparisons.

- open file
- parse file and remove unnecessary lines
- convert to pandas dataframe
- resample
- get daily statistics

## Next steps

- Look at how to calculate number of days with valid data
- Calculate uptime

In [1]:
import pandas as pd

def parse_file(energy_file_name):
    # read in raw CSV file for preprocessing
    energy_file = open(energy_file_name)
    # append data lines after header beginning with 'Timestamp'
    # dump lines from communications messages or empty lines and write to string
    file_string = ''
    omit_strings = ['Power Up', 'Communication Lost', 'Power Down', ',,,,,,,,,,,']
    # append data not matching omit_strings after sentinel is reached
    data_found = False
    for line in energy_file.readlines():
        if 'Timestamp' in line:
            data_found = True
        if not any(omit in line for omit in omit_strings) and data_found:
            file_string += line
    return file_string    

def create_data_frame(energy_file_name):
    # use pre-parsed string to read in data file to pandas and sort by date
    file_string = parse_file(energy_file_name)
    import io
    energy_data = pd.read_csv(io.StringIO(file_string),
                          index_col=0, 
                          parse_dates=[0], 
                          thousands=',')
    energy_data = energy_data.sort_index()
    return energy_data

def get_energy_stats(energy_data):
    energy = energy_data['kWh export']
    resampled = energy.resample('D').diff()
    return resampled.mean(), resampled.median()

In [2]:
energy_file_names = ["../data-timeseries/Data_Logs Ajau Agustus.csv",
                     "../data-timeseries/Data_Logs Atamali Agustus.csv",
                     "../data-timeseries/Data_Logs Ayapo Agustus.csv",
                     "../data-timeseries/Data_Logs ASEI July.csv",
                     "../data-timeseries/Data_Logs Kensio Agustus.csv"]

for energy_file_name in energy_file_names: 
    energy_data = create_data_frame(energy_file_name)
    print(energy_file_name, get_energy_stats(energy_data))

../data-timeseries/Data_Logs Ajau Agustus.csv (611.08185650485018, 623.0481108931781)
../data-timeseries/Data_Logs Atamali Agustus.csv (15.37019867111999, 16.787385937828503)
../data-timeseries/Data_Logs Ayapo Agustus.csv (95.387184750195217, 97.45942788562752)
../data-timeseries/Data_Logs ASEI July.csv (186.8583815028901, 213.20879022438567)
../data-timeseries/Data_Logs Kensio Agustus.csv (10.781886042778202, 9.85262661896104)
